## Deep Hybrid Swarm Intelligence IDS

This notebook implements an advanced hybrid swarm intelligence feature selection algorithm that integrates **Ant Colony Optimization (ACO)**, **Particle Swarm Optimization (PSO)**, **Artificial Bee Colony (ABC)**, and **Modified Wolf Predation Algorithm (MWPA)** within a single unified loop.

### 1. Setup & Data Acquisition

Install dependencies and load the NSL-KDD dataset.

In [ ]:

!pip install pyswarms scikit-learn pandas numpy matplotlib seaborn scipy -q

import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load from GitHub mirror
url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
cols_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDFeatureNames.txt"
features = pd.read_csv(cols_url, sep=":", header=None)[0].tolist() + ['target']
data = pd.read_csv(url, names=features)


### 2. Preprocessing

In [ ]:

# Encode and normalize
le = LabelEncoder()
for col in data.select_dtypes(include='object').columns:
    data[col] = le.fit_transform(data[col])
X = data.drop('target', axis=1).values
y = (data['target'] != 0).astype(int).values  # binary

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


### 3. Deep Hybrid Feature Selection

At each iteration, apply ACO pheromone update, PSO velocity-position update, ABC bee phases, and MWPA predation on a population of candidate feature masks.

In [ ]:

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def deep_hybrid_selection(X, y, pop_size=30, iters=50):
    n_feat = X.shape[1]
    # Initialize population of masks [0,1]
    pop = np.random.rand(pop_size, n_feat)
    pheromone = np.ones(n_feat)
    inertia = 0.9
    
    def binarize(arr):
        return 1/(1+np.exp(-10*(arr-0.5))) > 0.5
    
    def fitness(mask):
        sel = binarize(mask)
        if not sel.any(): return 1.0
        return 1 - cross_val_score(RandomForestClassifier(n_estimators=30), X[:, sel], y, cv=3).mean()

    for t in range(iters):
        # Evaluate
        fits = np.array([fitness(ind) for ind in pop])
        best_idx = np.argmin(fits)
        best = pop[best_idx]
        
        # ACO update
        pheromone = 0.8*pheromone + 0.2*(binarize(best).astype(float))
        prob = pheromone / pheromone.sum()
        
        for i in range(pop_size):
            # PSO update
            vel = inertia*pop[i] + 1.5*np.random.rand(n_feat)*(best - pop[i])
            pop[i] += vel
            # ABC local search
            k = np.random.choice([j for j in range(pop_size) if j!=i])
            phi = np.random.uniform(-1,1,n_feat)
            neighbor = pop[i] + phi*(pop[i] - pop[k])
            if fitness(neighbor) < fits[i]:
                pop[i] = neighbor
            # MWPA predation
            r1, r2 = np.random.rand(), np.random.rand()
            A = 2*(1 - t/iters)*r1 - (1 - t/iters)
            D = abs(2*r2*best - pop[i])
            pop[i] = best - A*(D**1.5)
            # ACO probabilistic mask
            mask = np.random.rand(n_feat) < prob
            pop[i] = np.where(mask, pop[i], pop[i])
        
        # Decrease inertia
        inertia *= 0.99

    final_mask = binarize(best)
    return np.where(final_mask)[0]

selected = deep_hybrid_selection(X_train, y_train)
print(f"Selected {len(selected)}/{X_train.shape[1]} features.")


### 4. Model Training & Evaluation

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

X_sel_train = X_train[:, selected]
X_sel_test  = X_test[:, selected]

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_sel_train, y_train)
y_pred = clf.predict(X_sel_test)
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, clf.predict_proba(X_sel_test)[:,1]))


### 5. Benchmark Analysis

Evaluate convergence on benchmark functions.

In [ ]:

from scipy.optimize import rosen
def sphere(x): return np.sum(x**2)
x0 = np.zeros(20)
print("Sphere:", sphere(x0))
print("Rosenbrock:", rosen(x0))


### 6. Conclusion

This deep hybrid approach tightly couples four SI algorithms, enhancing exploration and exploitation in feature selection.